In [34]:
import numpy as np  # http://www.numpy.org
import matplotlib.pyplot as plt   # http://matplotlib.org
import math as M  # https://docs.python.org/2/library/math.html
from sympy.solvers import solve
from sympy import Symbol

# Simple model including salinity

In [35]:
#### Constants #####
L_x = 1000000 #zonal extent [m]
L_y = 2000000 #meridional extent [m]
A = L_x*L_y #surface area of interior [m^2] Note that the interior is probably smaller than this
P = 2*L_x+2*L_y #Perimeter [m]

g = 9.81 #gravitational acceleration [m/s^2]
C_p = 3994 #specific heat capacity of sea water [J/(kg K)] source: Steffie's paper
alpha_T = 0.2 #thermal expansion coefficient [kg m^-3 C^-1] source: Steffie's paper
alpha_S = 0.2
c = 0.007 #bottom slope coefficient
s = 0.02 #average bottom slope around the perimeter of the marginal sea
S = 1
E = 1
S_1 = 1

#### Parameters ####
H = 1000 #sill height [m]
f_0 = 1.2e-4 #Coriolis parameter [s^-1]
Gamma = 20 #Restoring strength [W/(m^2 C)]

L = H/s #width of sloping topography of the boundary current [m]
T_1 = 8.87 #ingoing temperature [Celsius]
T_A = 2.8 #Atmospheric temperature [Celsius]
ro_0 = 1028  #density sea water [kg m^-3]. Range of 7 to 9 degrees of sea water
#h_1 = ... #thickness upper layer [m]

#### functions ####
epsilon = c*P/L #ratio of heat fluxed into interior by eddies compared to heat flux by advection into the basin in the inflowing boundary current [-]
mu = A*Gamma*f_0/(alpha_T*g*C_p*H**2*(T_1-T_A)) #[-]
gamma = 8*A*ro_0*f_0*S*alpha_S*E/(g*H**2*alpha_T**2*(T_1-T_A)**2)


In [41]:
#### Two main equations #####

x = Symbol('x')

#Interior temperature [Celsius]
solve(x**2 - M.sqrt(x**2 + gamma/epsilon)*x + 4*mu/epsilon*(x - 1), x)
T_0 = T_1 - x * (T_1 - T_A) 

x=2

#Interior salinity [g kg^-1]
Delta_S = 0.5*(x + M.sqrt(x**2 + gamma/epsilon)) #Still need to include +/-
S_0 = S_1 - Delta_S*alpha_T*(T_1-T_A)/alpha_S


TypeError: can't convert expression to float

In [6]:
#### Follow up computations ####

#Velocity in boundary current [m/s]
V_1 = g*H/(2*ro_0*f_0*L)*(alpha_T*(T_1-T_0) - alpha_S*(S_1-S_0))

#Outflowing salinity [g kg^-1]
S_out = S_1 + E*A*S_0/(V_1*H*L)

#Outflowing temperature [Celsius]
T_out = A*Gamma/(ro_0*C_p*V_1*H*L)*(T_0-T_A+P*L/A*(T_1-T_A))

#Densities -> not sure they are correct though..
ro_1 = ro_0*(1-alpha_T*(T_1-T_0) + alpha_S*(S_1-S_0))
ro_1 = ro_0*(1-alpha_T*(T_out-T_0) + alpha_S*(S_out-S_0))

#Downwelling/Medidional overturning strength at the sill [m^2/s]
W = g*H**2*(ro_1 - ro_out)/(4*ro_0*f_0)